<a href="https://colab.research.google.com/github/ozakiryota/kaggle/blob/main/titanic/titanic_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downlad and check datasets

Activate Kaggle API

In [ ]:
from google.colab import files
import os

jason_path = '/root/.kaggle/kaggle.json'

if not os.path.exists(jason_path):
    uploaded = files.upload()

    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))

    # Then move kaggle.json into the folder where the API expects to find it.
    !mkdir -p /root/.kaggle/ && mv kaggle.json /root/.kaggle/ && chmod 600 /root/.kaggle/kaggle.json

else:
    print(jason_path, "is already exists.")

/root/.kaggle/kaggle.json is already exists.


Download the datasets

In [ ]:
!pip install kaggle
!kaggle competitions download -c titanic

train.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


# Fitting and prediction

In [ ]:
import pandas as pd

from sklearn import svm

def extractData(csv_path):
    data = pd.read_csv(csv_path)
    data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
    data = pd.get_dummies(data)
    data = data.fillna(-1)

    return data

train_csv_path = '/content/train.csv'
eval_csv_path = '/content/test.csv'
save_csv_path = '/content/submission.csv'

train_data = extractData(train_csv_path)
train_inputs = train_data.drop(labels=["Survived"], axis=1)
train_labels = train_data["Survived"]

model = svm.SVC()
model.fit(train_inputs, train_labels)

eval_inputs = extractData(eval_csv_path)
predictions = model.predict(eval_inputs)

eval_indices_df = pd.read_csv(eval_csv_path)['PassengerId']
predictions_df = pd.Series(predictions, name='Survived')
result_df = pd.concat([eval_indices_df, predictions_df], axis=1)
result_df.to_csv(save_csv_path, index=False)

# Submission

In [ ]:
import sys

print("Submit the file? (y/n)")
ans = input()
while ans != 'y' and ans != 'n':
    print("Type 'y' or 'n'")
    ans = input()
if ans == 'y':
    !kaggle competitions submit -c titanic -f $save_csv_path -m "titanic_svm.ipynb"

Submit the file? (y/n)
y
100% 2.77k/2.77k [00:03<00:00, 752B/s]
Successfully submitted to Titanic - Machine Learning from Disaster